# Trees and dendrograms

## An example: the tree of life

<img src="https://raw.githubusercontent.com/ne1s0n/dataviz_python/main/resources/Phylogenetic%20tree%20of%20life.svg" width="800">

Source: [LUCA on wikipedia](https://raw.githubusercontent.com/ne1s0n/dataviz_python/main/resources/Phylogenetic%20tree%20of%20life.svg)
<br>




## What other people is doing

<img src="https://github.com/ne1s0n/dataviz_python/raw/main/resources/dendro_screenshot.png" width = "1000">

# Problem: the data/plot separation

Dendrograms are not simple graphs, and the kind of information to be plotted is complex, since it describe a hierarchy between groups (in fact dendrogram are typically used to represent results from [hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering) analyses).

This complicates a lot the topic, so much so that's difficul to achieve a good separation between the tool used to create the data and the tools used to plot the data. This means that you will have a hard time to apply our usual approach of "*load the numbers into a dataframe and then plot everything*".

In practical terms, the simplest solutions are achieved when the same tool that computes the dedrogram is also used for plotting. We'll see two solutions of this kind.


# Dendrograms with scipy

[Scipy](https://scipy.org/) is a library that provides:

> Fundamental algorithms for scientific computing in Python: optimization, integration, interpolation, eigenvalue problems, algebraic equations, differential equations, statistics and many other classes of problems 

(from Scipy homepage)



# Libraries 

All the libraries we are going to use in this lab

In [ ]:
# Libraries
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage

# A note of scipy version

In [ ]:
import scipy
print(scipy.__version__)

Just keep that in mind when reading the documentation

# Data 

Import the Canada dataset and keep only years (numeric) variables.


In [ ]:
#reading the data in
df_canada = pd.read_excel(
    'https://github.com/ne1s0n/dataviz_python/raw/main/resources/Canada.xlsx',
    sheet_name = 'Canada by Citizenship',  #the file contains three sheets
    skiprows = range(20), #skip the first twenty rows
    skipfooter = 2        #skip the last two rows
)

#renaming a column
df_canada.rename(columns = {'OdName':'Country'}, inplace = True)

#using Country as index
df_canada.set_index('Country', inplace = True)


#selectors for years only and for the first 30 countries to keep the plots small
years = df_canada.columns[8:42]
countries = df_canada.index[0:30]

#putting aside the continent column, we'll use it later
continents = df_canada.loc[countries, 'AreaName']

#subsetting the dataframe
df_canada = df_canada.loc[countries, years]

#let's check the result
print(df_canada.shape)
df_canada.head()

# Distance matrix, hierarchical clustering

Hierarchical clustering is an iterative process that starts from the original `N` observations and groups them in clusters, two at a time.

Let's comment an example of hierarchical clustering

<img src="https://raw.githubusercontent.com/ne1s0n/dataviz_python/main/resources/UPGMA_Dendrogram_Hierarchical.svg" width = 600>

By definition we see that the number of defined cluster is `N-1`, with `N` being the number of original observations.

To properly define the result of a hierarchical clustering we'll need the following pieces of data for each cluster:

* the two constituents, either original observations or other clusters
* the "distance" between the two constituents
* the total number of samples (this is not strictly required but for sure it's handy)

So we would expect the result to be a `(N-1)x4` matrix. This is not a casual choice, since it's exactly what's returned by [scipy.cluster.hierarchy.linkage()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) function, which we are going to use in few moments.

One key tool used in the clustering process is the concept of "distance matrix":

> If observations were N cities, a distance matrix would be a NxN symmetrical matrix reporting on each cell (i, j) the actual distance in Km between city i and city j

There are several ways of computing a distance matrix, depending on the distance function you adopt. The geographical example above would use the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) (ignoring Earth's curvature). Scipy supports many, many distances, as you can see in the documentation for funtion [`pdist()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html) (here "p" stands for paired).

A distance it's easily understood between observations. What about between clusters? There are several strategies (called `method`s in scipy lingo). Suppose that you are trying to compute the distance between cluster A and cluster B, knowing that cluster A is made putting together X and Y (being clusters or original observations). You may chose:

* `method=’single’` (aka "nearest point algorithm") : the distance is defined as the **minimum** distance between any points of cluster A and cluster B
* `method=’complete’` (aks Farthest Point Algorithm or Voor Hees Algorithm) : the distance is defined as the **maximum** distance between any points of cluster A and cluster B
* `method=’average’` : average distance between all possible pairing
* `method=’weighted’` (aka WPGMA) : given that `(X+Y) -> A` and we are trying to calculate `(A+B)`, you can distribute the operation and take the average `d(A, B) = [d(X, B) + d(Y, B)] / 2`
* `method=’centroid’` (aka UPGMC) : for each cluster you compute a "centroid", i.e. a new phantom entry that becomes representative for the cluster. You then use directly the centroid for distances. When putting together two clusters all elements are joined and a new centroid is recomputed
* `method=’median’` (aka WPGMC) : as above, but when joining clusters the new centroid is the average of the previous two centroids 
* `method=’ward’` (aka incremental algorithm) : it's [complicated](https://en.wikipedia.org/wiki/Ward%27s_method). Also usually creates compact, even-sized clusters, hence its popularity

In [ ]:
# compute the linkage matrix
Z = linkage(df_canada, method='ward', metric='euclidean')

In [ ]:
#can you guess what it will print?
Z.shape

Once we obtained the linkage matrix `Z` its just a matter of creating the plot using Scipy [`dendrogram()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html?highlight=dendrogram) function. 

In [ ]:
# Plot title
plt.title('Hierarchical Clustering Dendrogram')

# Plot axis labels
plt.xlabel('Countries')
plt.ylabel('distance (Euclidean + Ward)')

# Make the dendrogram
dendrogram(Z, labels=df_canada.index, leaf_rotation=90);

# Aesthetics

## Horizontal, bigger plot

In [ ]:
plt.figure(figsize = (14, 8))
dendrogram(Z, labels=df_canada.index, orientation = 'right');

## Threshold line

A common feature for dendrograms analysis. We impose a threshold and the clusters above it will have the same color.

We can also add a line using the same threshold with regular pyplot, just to have a visual reference.

In [ ]:
my_threshold = 5000

plt.figure(figsize = (14, 8))
dendrogram(Z, labels=df_canada.index, orientation = 'right',  color_threshold=my_threshold);
plt.axvline(x=my_threshold, c='grey', lw=1, linestyle='dashed')

## Custom colors

Cluster colors are set up using the dedicated function [`set_link_color_palette()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.set_link_color_palette.html)

The "above threshold" color has its own parameter in the `dendrogram()` call.

In [ ]:
from scipy.cluster.hierarchy import set_link_color_palette

my_threshold = 5000

plt.figure(figsize = (14, 8))
set_link_color_palette(['#FF00FF','#FFFF00', '#00FFFF'])
dendrogram(Z, labels=df_canada.index, orientation = 'right',  
           color_threshold=my_threshold,
           above_threshold_color='grey');
plt.axvline(x=my_threshold, c='grey', lw=1, linestyle='dashed')

 

---

# ASSIGNMENT! Custom dendrogram

Test a different distance/method and compare the results.

Also, there's too many levels! Find a way to limit the number of clustering levels to three.

---

In [ ]:
# Your solution here

# An alternative: Seaborn's clustermap

We have seen that Seaborn is able to plot heatmaps, i.e. a color coding of tabular (wide) data. The library also allows to add dendrograms to rows and columns, to show the relationship between variables.



In [ ]:
from seaborn import clustermap
clustermap(df_canada)

The default options has:

* dendrograms (and reordering) on both columns and rows
* method='average'
* metric='euclidean'

---

# ASSIGNMENT! Better clustermap

We want to improve Seaborn's clustermap on two aspects:

1. use the same Ward algorithm we used with Scipy and then check that the results are coherent
2. remove clustering for columns
3. add a small column color-coded that reports the continent to which each country belongs

For your convenience we already saved variable `continents`, and for colors you may use the following dictionary:

```
mycolors = {
    'Africa' : 'green',
    'Asia' : 'red',
    'Europe' : 'blue',
    'Latin America and the Caribbean' : 'purple',
    'Oceania' : 'orange'
}
```

You need to find the correct parameter in ['clustermap()'](https://seaborn.pydata.org/generated/seaborn.clustermap.html) function and understand how to use the dictionary.

---

In [ ]:
# Your solution here